## Base: Resultados de Análisis de Laboratorio Suelos en Colombia

Resultados del Servicio de Análisis de Suelos que el Laboratorio de Química y Física de Suelos de AGROSAVIA, presta al sector agropecuario, se enfoca en la EVALUACIÓN de la Fertilidad del Suelo, de la Salinidad y parámetros que Sirven como herramienta para la realización de planes de fertilización, Aplicación de enmiendas y adecuación de los terrenos para lograr producciones rentables.

Metadata: https://www.datos.gov.co/Agricultura-y-Desarrollo-Rural/Resultados-de-An-lisis-de-Laboratorio-Suelos-en-Co/ch4u-f3i5/about_data

Api: https://www.datos.gov.co/api/v3/views/ch4u-f3i5/query.json

In [1]:
import pandas as pd
import numpy as np
import requests
from sodapy import Socrata

In [2]:
# codigo de cargue tomado de "https://dev.socrata.com/foundry/www.datos.gov.co/ch4u-f3i5" que es la documentación de la API  

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("www.datos.gov.co", None)

# First 20000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("ch4u-f3i5", limit=95000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

In [3]:
results_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92738 entries, 0 to 92737
Data columns (total 32 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   secuencial                          92738 non-null  object
 1   fecha_de_an_lisis                   92738 non-null  object
 2   departamento                        92738 non-null  object
 3   municipio                           92736 non-null  object
 4   cultivo                             92738 non-null  object
 5   estado                              92738 non-null  object
 6   tiempo_de_establecimiento           92738 non-null  object
 7   topografia                          92738 non-null  object
 8   drenaje                             92738 non-null  object
 9   riego                               92738 non-null  object
 10  fertilizantes_aplicados             92738 non-null  object
 11  ph_agua_suelo                       92738 non-null  ob

In [4]:
results_df_cafe=results_df[results_df['cultivo']=='Café']
results_df_cafe.shape

(5080, 32)

#### Arreglo Fecha

In [5]:
# arreglo formato fecha
results_df_cafe['fecha_de_an_lisis']=pd.to_datetime(results_df_cafe['fecha_de_an_lisis'],format='%d/%m/%Y')
results_df_cafe['año_de_an_lisis']=results_df_cafe['fecha_de_an_lisis'].dt.year
results_df_cafe['mes_de_an_lisis']=results_df_cafe['fecha_de_an_lisis'].dt.month
results_df_cafe['fecha_de_an_lisis'].head()

C:\Users\vhmen\AppData\Local\Temp\ipykernel_15416\1867481610.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_df_cafe['fecha_de_an_lisis']=pd.to_datetime(results_df_cafe['fecha_de_an_lisis'],format='%d/%m/%Y')
C:\Users\vhmen\AppData\Local\Temp\ipykernel_15416\1867481610.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_df_cafe['año_de_an_lisis']=results_df_cafe['fecha_de_an_lisis'].dt.year
C:\Users\vhmen\AppData\Local\Temp\ipykernel_15416\1867481610.py:4: SettingWithCopyWarning: 
A valu

2   2014-08-22
4   2014-08-22
5   2014-08-22
6   2014-08-22
8   2014-08-22
Name: fecha_de_an_lisis, dtype: datetime64[ns]

In [6]:
results_df_cafe=results_df_cafe[(results_df_cafe['fecha_de_an_lisis']>='2015-01-01') & (results_df_cafe['fecha_de_an_lisis']<='2023-12-31')]
results_df_cafe.shape

(4333, 34)

#### Arreglo Divipola

In [7]:
results_df_cafe['departamento'].value_counts()
# dado que el proyecto se realizará para los departamentos de Cundinamarca y nariño se seleccionan estos
# dos departamentos y se complementa cruzando con la base divipola del dane para añadir el codigo y tener un cruce
# ajustado con las otras fuentes de información 
results_df_cafe=results_df_cafe[(results_df_cafe['departamento']=='CUNDINAMARCA') | (results_df_cafe['departamento']=='NARIÑO')]
results_df_cafe.shape

(2563, 34)

In [8]:
#cargar base divipola
divipola=pd.read_excel('DIVIPOLA_Municipios.xlsx'
                       , sheet_name='Municipios',nrows=1123, skiprows=10)
divipola = divipola.rename(columns={divipola.columns[0]: 'Código_departamento',divipola.columns[1]: 'departamento',
                       divipola.columns[2]: 'divipola',divipola.columns[3]: 'municipio'})
divipola=divipola.loc[(divipola['departamento']=='CUNDINAMARCA')|(divipola['departamento']=='NARIÑO'),['departamento','divipola','municipio']]
divipola

,departamento,divipola,municipio
458,CUNDINAMARCA,25001,AGUA DE DIOS
459,CUNDINAMARCA,25019,ALBÁN
460,CUNDINAMARCA,25035,ANAPOIMA
461,CUNDINAMARCA,25040,ANOLAIMA
462,CUNDINAMARCA,25053,ARBELÁEZ
...,...,...,...
775,NARIÑO,52786,TAMINANGO
776,NARIÑO,52788,TANGUA
777,NARIÑO,52835,SAN ANDRÉS DE TUMACO
778,NARIÑO,52838,TÚQUERRES


In [9]:
results_df_cafe=pd.merge(results_df_cafe,divipola, on=['departamento','municipio'], how='left')
results_df_cafe['divipola'] = results_df_cafe['divipola'].astype('Int64')
results_df_cafe.shape

(2563, 35)

### verificar valores nulos

In [10]:
results_df_cafe = results_df_cafe.replace('ND', np.nan)
results_df_cafe

C:\Users\vhmen\AppData\Local\Temp\ipykernel_15416\2192695840.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  results_df_cafe = results_df_cafe.replace('ND', np.nan)


,secuencial,fecha_de_an_lisis,departamento,municipio,cultivo,estado,tiempo_de_establecimiento,topografia,drenaje,riego,...,manganeso_disponible_olsen,zinc_disponible_olsen,boro_disponible,hierro_disponible_doble_acido,cobre_disponible_doble_acido,manganeso_disponible_doble_acido,zinc_disponible_doble_acido,año_de_an_lisis,mes_de_an_lisis,divipola
0,11687,2015-10-14,CUNDINAMARCA,LA VEGA,Café,Por establecer,No indica,Pendiente,Buen drenaje,No Tiene,...,8.484,3.498,0.331,NaN,NaN,NaN,NaN,2015,10,25402
1,11716,2015-11-10,CUNDINAMARCA,EL COLEGIO,Café,No indica,No indica,Pendiente,Buen drenaje,No indica,...,3.088,15.14,0.337,NaN,NaN,NaN,NaN,2015,11,25245
2,11771,2015-11-10,CUNDINAMARCA,EL COLEGIO,Café,No indica,De 1 a 5 años,Ondulado,Buen drenaje,No Tiene,...,2.626,5.266,1.000,NaN,NaN,NaN,NaN,2015,11,25245
3,11774,2015-11-10,CUNDINAMARCA,EL COLEGIO,Café,No indica,De 1 a 5 años,Ondulado,Buen drenaje,No Tiene,...,3.743,3.516,0.144,NaN,NaN,NaN,NaN,2015,11,25245
4,11775,2015-11-10,CUNDINAMARCA,EL COLEGIO,Café,No indica,De 1 a 5 años,Ondulado,Buen drenaje,No Tiene,...,5.644,3.516,0.251,NaN,NaN,NaN,NaN,2015,11,25245
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2558,86303,2023-04-24,CUNDINAMARCA,EL PEÑÓN,Café,Por establecer,De 0 a 1 año,Moderadamente ondulado,Regular drenaje,No Indica,...,1.78,3.49,0.13,NaN,NaN,NaN,NaN,2023,4,25258
2559,86304,2023-04-24,CUNDINAMARCA,EL PEÑÓN,Café,Por establecer,De 0 a 1 año,Pendiente moderada,Buen drenaje,No Indica,...,5.07,3.08,0.15,NaN,NaN,NaN,NaN,2023,4,25258
2560,86305,2023-04-24,CUNDINAMARCA,EL PEÑÓN,Café,Por establecer,De 0 a 1 año,Pendiente moderada,Regular drenaje,No Indica,...,3.62,3.06,0.09,NaN,NaN,NaN,NaN,2023,4,25258
2561,86306,2023-04-24,CUNDINAMARCA,EL PEÑÓN,Café,Por establecer,De 0 a 1 año,Moderadamente ondulado,Buen drenaje,No Indica,...,5.22,11.75,0.15,NaN,NaN,NaN,NaN,2023,4,25258


In [11]:
results_df_cafe.drop(['hierro_disponible_doble_acido', 'cobre_disponible_doble_acido',
                      'manganeso_disponible_doble_acido','zinc_disponible_doble_acido'], axis=1, inplace=True)
results_df_cafe=results_df_cafe.dropna(subset=['municipio'])

#### Cambiar formato para realizar agregación

In [12]:
results_df_cafe['ph_agua_suelo'] = results_df_cafe['ph_agua_suelo'].astype(float)

results_df_cafe['hierro_disponible_olsen'] = results_df_cafe['hierro_disponible_olsen'].str.replace('[<>]', '', regex=True)
results_df_cafe['hierro_disponible_olsen'] = results_df_cafe['hierro_disponible_olsen'].astype(float)

results_df_cafe['zinc_disponible_olsen'] = results_df_cafe['zinc_disponible_olsen'].str.replace('[<>]', '', regex=True)
results_df_cafe['zinc_disponible_olsen'] = results_df_cafe['zinc_disponible_olsen'].astype(float)

results_df_cafe['boro_disponible'] = results_df_cafe['boro_disponible'].str.replace('[<>]', '', regex=True)
results_df_cafe['boro_disponible'] = results_df_cafe['boro_disponible'].astype(float)

results_df_cafe['manganeso_disponible_olsen'] = results_df_cafe['manganeso_disponible_olsen'].str.replace('[<>]', '', regex=True)
results_df_cafe['manganeso_disponible_olsen'] = results_df_cafe['manganeso_disponible_olsen'].astype(float)

results_df_cafe['acidez_kcl'] = results_df_cafe['acidez_kcl'].str.replace('[<>]', '', regex=True)
results_df_cafe['acidez_kcl'] = results_df_cafe['acidez_kcl'].astype(float)

results_df_cafe['cobre_disponible'] = results_df_cafe['cobre_disponible'].str.replace('[<>]', '', regex=True)
results_df_cafe['cobre_disponible'] = results_df_cafe['cobre_disponible'].astype(float)

C:\Users\vhmen\AppData\Local\Temp\ipykernel_15416\2020515650.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_df_cafe['ph_agua_suelo'] = results_df_cafe['ph_agua_suelo'].astype(float)
C:\Users\vhmen\AppData\Local\Temp\ipykernel_15416\2020515650.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_df_cafe['hierro_disponible_olsen'] = results_df_cafe['hierro_disponible_olsen'].str.replace('[<>]', '', regex=True)
C:\Users\vhmen\AppData\Local\Temp\ipykernel_15416\2020515650.py:4: SettingWithCo

#### Sacar base final

In [13]:
results_df_cafe_agg = results_df_cafe.groupby(['año_de_an_lisis', 'mes_de_an_lisis','divipola']).agg({
    'departamento': 'max',
    'municipio': 'max',
    'cobre_disponible': 'mean',
    'hierro_disponible_olsen': 'mean',
    'zinc_disponible_olsen': 'mean',
    'boro_disponible': 'mean',
    'manganeso_disponible_olsen': 'mean',
    'acidez_kcl': 'mean',
    'ph_agua_suelo':'mean'
}).reset_index()

In [14]:
results_df_cafe_agg

,año_de_an_lisis,mes_de_an_lisis,divipola,departamento,municipio,cobre_disponible,hierro_disponible_olsen,zinc_disponible_olsen,boro_disponible,manganeso_disponible_olsen,acidez_kcl,ph_agua_suelo
0,2015,2,25095,CUNDINAMARCA,BITUIMA,2.90000,267.000000,6.300000,0.197000,5.300000,NaN,5.540000
1,2015,2,25506,CUNDINAMARCA,VENECIA,2.93725,388.915833,5.054125,0.225167,6.528542,1.104923,5.457917
2,2015,2,25524,CUNDINAMARCA,PANDI,3.10500,387.465000,5.659950,0.218600,7.074750,0.962000,5.358950
3,2015,2,25718,CUNDINAMARCA,SASAIMA,3.30000,394.000000,1.100000,0.090000,3.300000,2.826000,4.769000
4,2015,3,25095,CUNDINAMARCA,BITUIMA,1.98995,160.155000,5.379400,0.247650,5.169300,3.702182,5.626850
...,...,...,...,...,...,...,...,...,...,...,...,...
420,2023,12,25823,CUNDINAMARCA,TOPAIPÍ,4.33000,547.940000,1.830000,0.150000,1.370000,9.070000,4.320000
421,2023,12,25862,CUNDINAMARCA,VERGARA,2.68000,316.680000,8.150000,0.200000,6.850000,4.390000,4.890000
422,2023,12,25878,CUNDINAMARCA,VIOTÁ,1.00000,159.410000,1.000000,0.100000,1.000000,12.400000,4.740000
423,2023,12,52110,NARIÑO,BUESACO,1.00000,96.210000,2.600000,0.180000,7.270000,NaN,5.960000


In [15]:
results_df_cafe_agg.to_excel('Base_AnalisisLabSuelo_municipio_mes.xlsx', sheet_name='Base', index=False)